In [1]:
import os, sys, glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import hist
from hist import Hist
import pickle
import mplhep as hep
import math
from tabulate import tabulate
import itertools
import warnings
from plot_utils import *

warnings.filterwarnings('ignore')
hep.style.use("CMS")
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams['figure.facecolor'] = 'white'

## Notebook Parameters
`tag`: _str_, tag used to identify which .pkl files to import. Will select `infile_names` to have `<tag>.pkl`.<br>
`output_label`: _str_, saves plots as `<output_label>+<plot_name>.<file_ext>`.<br>
`file_ext`: _str_, see above.<br>
`infile_names`: _array of str_: full paths of .pkl files.<br>

In [2]:
# parameters
input_label = 'test'
output_label = '/home/submit/lavezzo/public_html/SUEP/analysis/test/'
file_ext = '.png'

# input .pkl files
#plotDir = '/work/submit/lavezzo/SUEP/pu/SUEPCoffea_dask/plotting/'
plotDir = '/work/submit/lavezzo/SUEP/plotting/'
infile_all = [plotDir+'/outputs/'+dataset for dataset in os.listdir(plotDir+'/outputs/')]

# some predefined samples to choose from
infile_names_2018 = [f for f in infile_all if '20UL18' in f or 'UL2018' in f]
infile_names_2018_data = [f for f in infile_all if 'UL2018' in f]
infile_names_2017 = [f for f in infile_all if '20UL17' in f or 'UL2017' in f]
infile_names_2016 = [f for f in infile_all if ('20UL16MiniAODv2' in f or 'UL2016' in f) \
                     and ('HIPM_UL' not in f)]
infile_names_2016_apv = [f for f in infile_all if ('20UL16MiniAODAPVv2' in f or 'HIPM_UL2016' in f)]
infile_names_SUEP = [f for f in infile_all if ('SUEP-m' in f)]


# assign your choice to infile_names
infile_names = infile_names_2018 + infile_names_SUEP
includers = [input_label+'.pkl']
excluders = ['.root']
infile_names = [f for f in infile_names if all([i in f for i in includers])]
infile_names = [f for f in infile_names if not any([e in f for e in excluders])]

The `plots` dictionary is used to store all the plots imported from the pickle files. Dimensions: (real or MC) x (plot label) <br>
The `data` nested dictionary stores the number of events and variances in each region for real data and MC data when the plots are made, and is used to make the comparison table. Dimensions: (real or MC) x (plot label) x (value or error on the value)

In [3]:
# dimensions: (data or MC) x (plot label) x (value or error on the value)
stats = nested_dict(4,float)

# dimensions: (data or MC) x (plot label)
plots = {}

# create output dir
if not os.path.isdir(output_label): 
    print("Creating", output_label)
    os.mkdir(output_label)

# load file(s)
for infile_name in infile_names:
    if not os.path.isfile(infile_name): 
        print("WARNING:",infile_name,"doesn't exist")
        continue
    elif input_label not in infile_name: continue
    elif ".pkl" not in infile_name: continue
    elif "QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-pilot" in infile_name: continue
    
    # sets the lumi based on year
    if ('20UL16MiniAODv2' in infile_name):
        lumi = lumis['2016']
    if ('20UL17MiniAODv2' in infile_name):
        lumi = lumis['2017']
    if ('20UL16MiniAODAPVv2' in infile_name):
        lumi = lumis['2016_apv']
    if ('20UL18' in infile_name):
        lumi = lumis['2018']
    if 'SUEP-m' in infile_name:
        lumi = lumis['2018']
    if 'JetHT+Run' in infile_name:
        lumi = 1
                                
    # exclude low bins
    if '50to100' in infile_name: continue
    if '100to200' in infile_name: continue
    if '200to300' in infile_name: continue
    if '300to500' in infile_name: continue
    if '500to700' in infile_name: continue
    if '700to1000' in infile_name: continue
    
    if '15to30' in infile_name: continue
    if '30to50' in infile_name: continue
    if '50to80' in infile_name: continue
    if '80to120' in infile_name: continue
    if '120to170' in infile_name: continue
    if '170to300' in infile_name: continue
    
    # plots[sample] sample is set here
    if 'QCD_Pt' in infile_name:
        sample = 'QCD_Pt'
        
        # include this block to import the QCD bins individually
        temp_sample = infile_name.split('/')[-1].split('.pkl')[0]
        plots[temp_sample] = openpkl(infile_name)
        for plot in list(plots[temp_sample].keys()):
            plots[temp_sample][plot] = plots[temp_sample][plot]*lumi
        
    elif 'QCD_HT' in infile_name:
        sample = 'QCD_HT'
        
        # include this block to import the QCD bins individually
        temp_sample = infile_name.split('/')[-1].split('.pkl')[0]
        plots[temp_sample] = openpkl(infile_name)
        for plot in list(plots[temp_sample].keys()):
            plots[temp_sample][plot] = plots[temp_sample][plot]*lumi
        
    elif 'JetHT+Run' in infile_name: 
        sample = 'data'
        
    elif 'SUEP-m' in infile_name:
        sample = infile_name.split('/')[-1].split('+')[0]

    if sample not in list(plots.keys()): 
        plots[sample] = openpkl(infile_name)
        for plot in list(plots[sample].keys()):
            plots[sample][plot] = plots[sample][plot]*lumi
    else:
        plotsToAdd = openpkl(infile_name) 
        for plot in list(plotsToAdd.keys()):
            plots[sample][plot]  = plots[sample][plot] + plotsToAdd[plot]*lumi
                
# calculate D_exp region after combining samples
# for selection in ['IRM', 'ML', 'CL']:
#     for sample in list(plots.keys()):
#         plots[sample]['D_exp_' + selection] = D_expect(plots[sample],selection)
#         plots[sample]['D_exp_var2_' + selection] = D_expect_var2(plots[sample],selection)

These are all the samples loaded.

In [4]:
keys = list(plots.keys())
keys.sort()
for k in keys: print(k)

QCD_HT
QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM_test
QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM_test
QCD_HT2000toInf_TuneCP5_PSWeights_13TeV-madgraph-pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM_test
SUEP-m1000-darkPho
SUEP-m1000-darkPhoHad
SUEP-m1000-generic
SUEP-m125-darkPho
SUEP-m125-darkPhoHad
SUEP-m125-generic
SUEP-m400-darkPho
SUEP-m400-darkPhoHad
SUEP-m400-generic
SUEP-m750-darkPho
SUEP-m750-darkPhoHad
SUEP-m750-generic
data


## 1D Plots
Plot some of the 1D histograms for any number of samples in the same plot.<br>
Parameters:<br>
`plot_labels`: _list of str_, which plots to show.<br>
`samples`: _list of str_, which samples to include in the plot.

In [ ]:
plot_labels = ['A_SUEP_S1_CL']
samples = ['QCD_HT']
samples += ['SUEP-m125-generic', 'SUEP-m400-generic', 'SUEP-m750-generic', 'SUEP-m1000-generic']

for plot_label in plot_labels:
    for sample in samples: 
        hep.histplot(plots[sample][plot_label][::1j], 
                     yerr=False, label=sample) 
    
    # plt.ylim(1e4,1e6)
    #plt.xlim(0,150)
    plt.legend()
    plt.yscale("log")
    plt.savefig(output_label + plot_label + file_ext, bbox_inches='tight')
    plt.show()

## 2D Plots
Plot some of the 2D histograms for any number of samples in the same plot.<br>
Parameters:<br>
`plot_label`: _str_, which plot to show.<br>
`samples`: _list of str_, which samples to include in the plot.

In [ ]:
plot_label = '2D_SUEP_S1_SUEP_nconst_IRM'
samples = ['data']
#samples += ['SUEP-m125-generic', 'SUEP-m400-generic', 'SUEP-m750-generic', 'SUEP-m1000-generic']
draw_lines = False
line_x = 0.5
line_y = 100

for sample in samples:
    
    if plots[sample][plot_label].sum().value == 0:
        print("Histogram empty for sample", sample)
        continue

    hep.hist2dplot(plots[sample][plot_label], 
               norm=matplotlib.colors.LogNorm(),
               label=sample) 
    
    # draw lines to visualize ABCD regions
    if draw_lines:
        plt.plot([0,10000], [line_y,line_y], color='red', linestyle='dashed')
        plt.plot([line_x,line_x], [0,1000], color='red', linestyle='dashed')
    
    plt.savefig(output_label + plot_label+ "_"+sample+file_ext)
    plt.show()

## Ratio Plots
Make ratio plots of different datasets.<br>
Parameters:<br>
`selection`: _str_, which selection to use, e.g. 'ch', 'mult'.<br>
`plot_labels`: _list of str_, which plots to make.

In [ ]:
selection = 'CL'
# plot_labels = ['A_ht', 'A_ngood_fastjets', 'A_PV_npvs', 'A_ngood_ak4jets']
plot_labels = ['C_SUEP_nconst']
density = False

for plot_label in plot_labels:
    
    plot_label += '_'+selection

    h1 = plots['QCD_HT'][plot_label][::1j]
    h2 = plots['data'][plot_label][::1j]
    
    if density:
        h1 /= h1.sum().value
        h2 /= h2.sum().value
        
    fig = plt.figure(figsize=(10, 8))
    _ = h1.plot_ratio(h2,
                     rp_num_label='QCD - HT Binned',
                     rp_denom_label='data')
 
    fig.axes[0].set_yscale("log")
    #fig.axes[0].set_xlim(0, 2000)
    #fig.axes[0].set_ylim(1e-5, 1e7)
    fig.axes[1].set_ylim(0.4,1.6)
    fig.axes[1].set_ylabel('MC/data')
    plt.savefig(output_label + plot_label + file_ext, bbox_inches='tight')
    plt.show()

### SUEP vs. ISR Comparison

In [ ]:
plot_label = 'SUEP_pt_mean_scaled'

# QCD hist for ABCD region
h1 = plots['QCD_HT'][plot_label+"_CL"][::2j]
h2 = plots['QCD_HT'][plot_label.replace("SUEP", "ISR") +"_CLi"][::2j]

h1 /= h1.sum().value
h2 /= h2.sum().value

fig = plt.figure(figsize=(10, 8))
hep.histplot(h1, yerr=False, label='SUEP') 
hep.histplot(h2, yerr=False, label='ISR') 

fig.axes[0].set_yscale("log")
# fig.axes[0].set_xlabel("pT")
#fig.axes[0].set_xlim(0, 2000)
#fig.axes[0].set_ylim(1e-5, 1e7)
# fig.axes[1].set_ylim(0.4,1.6)
plt.legend(bbox_to_anchor=(1, 0.5))
plt.savefig(output_label + plot_label + file_ext, bbox_inches='tight')
plt.show()

### Ratio Plots: QCD D Region Closure
Make ratio plots to check ABCD closure in the D region.<br>
Parameters:<br>
`selection`: _str_, which selection to use, e.g. 'ch', 'mult'.<br>

In [ ]:
hQCD_MC_D_obs = plots['QCD_MC']['D_obs_IRM']
hQCD_MC_D_exp = plots['QCD_MC']['D_exp_IRM']

_ = hQCD_MC_D_obs.plot_ratio(hQCD_MC_D_exp,
                 rp_num_label='D observed',
                 rp_denom_label='D expected')

plt.xlabel("Sphericity")
plt.ylim(0.5,1.5)
plt.savefig(output_label + "_D_closure" + file_ext, bbox_inches='tight')
plt.show()

## Automatic Plotting
Make all plots in the pkl and save them automatically.<br>

### All 1D and 2D Plots

In [ ]:
outdir = 'allplots/'

plot_labels = [k for k in list(plots['QCD_HT'].keys())]
if not os.path.isdir(output_label+outdir): os.system("mkdir " + str(output_label+outdir))

for plot_label in plot_labels:
    
    # 2D plots
    if '2D' in plot_label:
        
        print(plot_label, end=", ")
        
        for sample in ['QCD_HT', 'data']:
                   #'SUEP-m125-generic', 'SUEP-m400-generic', 'SUEP-m750-generic', 'SUEP-m1000-generic']:

            if plots[sample][plot_label].sum().value == 0:
                print("Histogram empty for sample", sample, end=", ")
                continue

            hep.hist2dplot(plots[sample][plot_label], 
                       norm=matplotlib.colors.LogNorm(),
                       label=sample) 
            plt.savefig(output_label + outdir + plot_label+ "_"+sample+file_ext, bbox_inches='tight')
            plt.close()
        
    # 1D variables
    elif '2D' not in plot_label:
    
        print(plot_label, end=", ")
        
        # plot everything
        for sample in ['QCD_HT', 'data']:
                      # 'SUEP-m125-generic', 'SUEP-m400-generic', 'SUEP-m750-generic', 'SUEP-m1000-generic']:
            hep.histplot(plots[sample][plot_label], yerr=True, label=sample) 
        plt.legend()
        plt.yscale("log")
        plt.savefig(output_label + 'allplots/' + plot_label+ "_all"+file_ext, bbox_inches='tight')
        plt.close()

    print()
print("All done")

### All Ratios: 1D Plots
For each 1D histogram, make a ratio plot between two samples.

In [ ]:
plot_labels = list(plots['QCD_HT'].keys())
density = False
combos = itertools.combinations(['QCD_HT', 'data'], 2)

for l1, l2 in combos:
    
    outdir = l1+"_"+l2+"/"
    print(outdir)

    if not os.path.isdir(output_label+outdir): 
        os.system("mkdir " + str(output_label+outdir))

    for plot_label in plot_labels:

        if '2D' in plot_label: continue
        if "D_obs" in plot_label: continue

        print(plot_label, end=", ")
        h1 = plots[l1][plot_label]
        
        if h1.sum().value == 0: continue
        if density: h1 /= h1.sum().value
        h2 = plots[l2][plot_label]
        if h2.sum().value == 0: continue
        if density: h2 /= h2.sum().value
    
        fig = plt.figure(figsize=(10, 8))
        _ = h1.plot_ratio(h2,
                         rp_num_label=l1,
                         rp_denom_label=l2)
        fig.axes[0].set_yscale("log")
        fig.axes[1].set_ylim(0.4,1.6)
        plt.savefig(output_label + outdir + plot_label +file_ext, bbox_inches='tight')
        plt.close()

        print()
    print("All done")

### All Ratios: Combined Regions

In [5]:
plot_labels = list(plots['QCD_HT'].keys())
combos = itertools.combinations(['QCD_HT', 'data'], 2)

for l1, l2 in combos:
    
    outdir = l1+"_"+l2+"_combinedRegions/"
    print(outdir)

    if not os.path.isdir(output_label+outdir): 
        os.system("mkdir " + str(output_label+outdir))

    for plot_label in plot_labels:

        if "A_" not in plot_label: continue
        print(plot_label, end=", ")
        
        h1 = plots['QCD_HT'][plot_label.replace("A_", "")]
        h2 = plots['data'][plot_label.replace("A_", "")]
        if h1.sum().value == 0 or h2.sum().value == 0:
            print("Empty", end=", ")
            continue
            
        x = 'ABCDEFGH'
        y = 'ADGBEHCF'
        _ = plot_ratio_regions(plots, plot_label, 'QCD_HT', 'data', x)
        plt.savefig(output_label + outdir + plot_label +file_ext, bbox_inches='tight')
        plt.close()

QCD_HT_data_combinedRegions/
A_ht_IRM, A_ht_tracker_IRM, A_ntracks_IRM, A_ngood_fastjets_IRM, A_PV_npvs_IRM, A_Pileup_nTrueInt_IRM, Empty, A_ngood_ak4jets_IRM, A_SUEP_nconst_IRM, A_SUEP_pt_IRM, A_SUEP_pt_avg_IRM, A_SUEP_pt_avg_b_IRM, A_SUEP_pt_mean_scaled_IRM, A_SUEP_eta_IRM, A_SUEP_phi_IRM, A_SUEP_mass_IRM, A_SUEP_S1_IRM, A_SUEP_girth, Empty, A_SUEP_rho0_IRM, A_SUEP_rho1_IRM, A_ht_ML, Empty, A_ht_tracker_ML, Empty, A_ntracks_ML, Empty, A_ngood_fastjets_ML, Empty, A_PV_npvs_ML, Empty, A_Pileup_nTrueInt_ML, Empty, A_ngood_ak4jets_ML, Empty, A_resnet_SUEP_pred_ML, Empty, A_ht_CL, A_ht_tracker_CL, A_ntracks_CL, A_ngood_fastjets_CL, A_PV_npvs_CL, A_Pileup_nTrueInt_CL, Empty, A_ngood_ak4jets_CL, A_SUEP_nconst_CL, A_SUEP_pt_CL, A_SUEP_pt_avg_CL, A_SUEP_pt_avg_b_CL, A_SUEP_pt_mean_scaled_CL, A_SUEP_eta_CL, A_SUEP_phi_CL, A_SUEP_mass_CL, A_SUEP_S1_CL, A_SUEP_rho0_CL, A_SUEP_rho1_CL, A_ht_CLi, Empty, A_ht_tracker_CLi, Empty, A_ntracks_CLi, Empty, A_ngood_fastjets_CLi, Empty, A_PV_npvs_CLi, Empt

## Binned QCD Plots
Make plots with the QCD separated by bins. This won't work if QCD has been marged. <br>
Parameters:<br>
`selection`: _str_, which selection to use, e.g. 'ch', 'mult'.<br>
`plot_labels`: _list_, which plots you want to make.<br>
`identifier`: _str_, how to identify the QCD samples, e.g. QCD_HT or QCD_Pt.

In [ ]:
plot_labels = ['A_PV_npvs_IRM']
identifier = 'QCD_HT'

for plot_label in plot_labels:
    
    # get hists
    hlist, labels = [], []
    for key in plots.keys():
        if identifier == key: continue
        if identifier in key:
            hlist.append(plots[key][plot_label])
            key = key.split('_Tune')[0]
            labels.append(key)
                        
    # not sure why, but they NEED to be color coded appropriately
    vals = []
    for key in labels:
        if 'Pt' in identifier: keyslice = key.split(identifier+"_")[1]
        if 'HT' in identifier: keyslice = key.split(identifier)[1]
        if len(keyslice) == 0: continue
        keyslice = keyslice.split("to")[0]
        val = int(keyslice)
        vals.append(val)
    hlist_sorted, labels_sorted = [], []
    for arg in np.argsort(vals)[::-1]:
        hlist_sorted.append(hlist[arg])
        labels_sorted.append(labels[arg])
        
    cmap = plt.cm.jet(np.linspace(0, 1, len(labels_sorted)))
    
    hep.histplot(hlist_sorted, yerr=True, stack=True, histtype ='fill',
                 label=labels_sorted, color=cmap)
    
plt.legend(fontsize=14, framealpha=1, facecolor='white', shadow=True, bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim(1e-8,1e8)
plt.yscale("log")
plt.savefig(output_label + plot_label + file_ext, bbox_inches='tight')

## ABCD
Make the ABCD regions by hand from one of the 2D plots.<br>
Notation: `hist[v1:v2::sum]` sums along an axis from v1 to v2 (leave either blank to extend the summation bounds arbitrarily).

### ABCD Independent Variables Check
1D slices of 2D plots, usually used to check the independence of the ABCD method's two variables on the QCD sample. This won't work if QCD has been binned by pt. <br>
N.B.: Imaginary numbers are used to access the bin value; using real numbers would access the bin number (see hist documentation for more info).<br>
Parameters:<br>
`plot_label`:_str_, which plot to analyze.<br>
`sample`: _str_, which sample to analyze.<br>
`variables`: _str_, variables being sliced, used to pick the slices in the loop.<br>
`slices`: _list of lists of floats_, bins of values to be sliced. e.g. for equal slices use np.linspace(start, end, number of bins), if not can specify as [[0,10],[20,40],[40,50],...].

In [ ]:
# parameters
plot_label = '2D_SUEP_nconst_SUEP_pt_avg_b_CL'
sample = 'QCD_HT'
variables = ['S1', '# Tracks']

for i in range(2):
    
    slice_label = variables[(i+1)%2]
    
    if slice_label == '# Tracks': slices = [[0j,40j],[40j,80j],[80j,120j],[120j,140j],[140j,160j],[160j,500j]]
    elif slice_label == '# Const': slices = [[0j,20j],[20j,40j],[40j,60j],[60j,80j],[80j,100j]]
    elif slice_label == 'pT Avg': slices = [[0.0j,20j],[20j,40j],[40j,60j],[60j,80j],[80j,1000j]]
    elif slice_label == 'pT Avg b': slices = [[0.0j,2j],[2j,4j],[4j,6j],[6j,10j],[10j,1000j]]
    elif slice_label == 'pT Mean Scaled': slices = [[0.0j, 0.1j],[0.1j,0.2j],[0.2j,0.4j],[0.4j,0.6j],[0.6j, 1.0j]]
    elif slice_label == 'Classifier Output': slices = [[0.0j,0.05j],[0.05j,0.1j],[0.1j,0.2j],[0.2j,0.4j],[0.4j,0.6j],[0.6j,0.8j],[0.8j,0.9j], [0.9j,1.0j]]
    elif slice_label == 'S1': slices = [[0.0j,0.2j],[0.2j,0.4j],[0.4j,0.6j],[0.6j,0.8j],[0.8j,1.0j]]

    for j in range(len(slices)):
        val_lo = slices[j][0]
        val_up = slices[j][1]
        
        if i==0: 
            h = plots[sample][plot_label][::2j,val_lo:val_up:sum]
        elif i==1:
            h = plots[sample][plot_label][val_lo:val_up:sum, ::2j]
            h = rebin(h, [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.44, 0.48, 0.52, 0.6, 0.7, 0.8, 1.0])

        hep.histplot(h,
                     yerr=True, density=True, 
                     label=slice_label+" [{0},{1}]".format(val_lo.imag, val_up.imag))    
    
    plt.legend()
    #plt.ylim(1e-6,1e0)
    plt.yscale('log')
    plt.ylabel("Normalized Events")
    plt.xlabel(variables[i])
    plt.savefig(output_label + plot_label + "_" + slice_label.replace("#","n").replace(" ","_") + "_SLICES" + file_ext,  bbox_inches='tight')
    plt.show()

### D Region Closure
Check closure in D region for one sample.

In [ ]:
plot_label = '2D_SUEP_S1_ntracks_IRM'
abcd = plots['QCD_HT'][plot_label]
A = abcd[0.35j:0.5j:sum,0.0j:100.0j:2j]
B = abcd[0.35j:0.5j:sum,100.0j:300.0j:2j]
C = abcd[0.5j:1.0j:sum,0.0j:100.0j:2j]
D = abcd[0.5j:1.0j:sum,100.0j:300.0j:2j]
# A = abcd[20j:80j:2j,5j:100j:sum]
# B = abcd[80j:300j:2j,5j:100j:sum]
# C = abcd[20j:80j:2j,0.0j:5j:sum]
# D = abcd[80j:300j:2j,0.0j:5j:sum]
# A = abcd[0.35j:0.5j,0.3j:50.0j:sum]
# B = abcd[0.35j:0.5j,0.0j:0.3j:sum]
# C = abcd[0.5j:1.0j,0.3j:50j:sum]
# D = abcd[0.5j:1.0j,0.0j:0.3j:sum]
D_exp = B * C.sum().value / A.sum().value

print(D_exp.sum().value/D.sum().value)

fig = plt.figure(figsize=(10, 8))
main, sub = D.plot_ratio(D_exp,
                rp_num_label='SR observed',
                rp_denom_label='SR expected',
                rp_uncert_draw_type="bar")
fig.axes[0].set_yscale("log")
fig.axes[0].set_xlim(150,300)
fig.axes[1].set_xlabel("# Const")
fig.axes[1].set_ylim(0.4,1.6)
plt.savefig(output_label+"ABCD_"+plot_label+file_ext)

### 6 Regions

In [ ]:
plot_label = '2D_SUEP_S1_SUEP_pt_mean_scaled_IRM'
abcd = plots['QCD_HT'][plot_label]
# A = abcd[20j:40j:2j,5j:100j:sum]
# B = abcd[40j:80j:2j,5j:100j:sum]
# C = abcd[80j:300j:2j,5j:100j:sum]
# D = abcd[20j:40j:2j,0.0j:5j:sum]
# E = abcd[40j:80j:2j,0.0j:5j:sum]
# F = abcd[80j:300j:2j,0.0j:5j:sum]
# A = abcd[0.35j:0.5j:sum,20j:40j:2j]
# B = abcd[0.35j:0.5j:sum,40j:80j:2j]
# C = abcd[0.35j:0.5j:sum,80j:300j:2j]
# D = abcd[0.5j:1.0j:sum,20j:40j:2j]
# E = abcd[0.5j:1.0j:sum,40j:80j:2j]
# F = abcd[0.5j:1.0j:sum,80j:300j:2j]
# A = abcd[0.35j:0.5j:sum,70.0j:100j:2j]
# B = abcd[0.35j:0.5j:sum,100j:150j:2j]
# C = abcd[0.35j:0.5j:sum,150j:300j:2j]
# D = abcd[0.5j:1.0j:sum,70.0j:100j:2j]
# E = abcd[0.5j:1.0j:sum,100j:150j:2j]
# F = abcd[0.5j:1.0j:sum,150j:300j:2j]
# A = abcd[0.35j:0.5j:sum,7.0j:100j:2j]
# B = abcd[0.35j:0.5j:sum,4.0j:7.0j:2j]
# C = abcd[0.35j:0.5j:sum,0.0j:4.0j:2j]
# D = abcd[0.5j:1.0j:sum,7.0j:100j:2j]
# E = abcd[0.5j:1.0j:sum,4.0j:7.0j:2j]
# F = abcd[0.5j:1.0j:sum,0.0j:4.0j:2j]
# A = abcd[0.35j:0.45j:2j,4j:100j:sum]
# B = abcd[0.45j:0.55j:2j,4j:100j:sum]
# C = abcd[0.55j:1.0j:2j,4j:100j:sum]
# D = abcd[0.35j:0.45j:2j,0.0j:4j:sum]
# E = abcd[0.45j:0.55j:2j,0.0j:4j:sum]
# F = abcd[0.55j:1.0j:2j,0.0j:4j:sum]
# A = abcd[50.0j:100j:2j,4.0j:100j:sum]
# B = abcd[100j:150j:2j,4.0j:100j:sum]
# C = abcd[150j:300j:2j,4.0j:100j:sum]
# D = abcd[50.0j:100j:2j,0.0j:4.0j:sum]
# E = abcd[100j:150j:2j,0.0j:4.0j:sum]
# F = abcd[150j:300j:2j,0.0j:4.0j:sum]

A = abcd[0.35j:0.4j:2j,0.3j:100j:sum]
B = abcd[0.4j:0.5j:2j,0.3j:100j:sum]
C = abcd[0.5j:1.0j:2j,0.3j:100j:sum]
D = abcd[0.35j:0.4j:2j,0.0j:0.3j:sum]
E = abcd[0.4j:0.5j:2j,0.0j:0.3j:sum]
F = abcd[0.5j:1.0j:2j,0.0j:0.3j:sum]

F_exp = C * E.sum().value**2 * A.sum().value / (B.sum().value**2 * D.sum().value)

fig = plt.figure(figsize=(10, 8))
main, sub = F.plot_ratio(F_exp,
                rp_num_label='SR observed',
                rp_denom_label='SR expected',
                rp_uncert_draw_type="bar")
fig.axes[0].set_yscale("log")
fig.axes[0].set_xlim(0.5,1)
fig.axes[1].set_ylim(0.4,1.6)
plt.savefig(output_label+"ABCD_6regions_"+plot_label+file_ext)

### 9 Regions

In [ ]:
plot_label = '2D_SUEP_S1_SUEP_nconst_CL'
abcd = plots['QCD_HT'][plot_label]

# A = abcd[20j:40j,7.0j:100j:sum]
# B = abcd[20j:40j,4.0j:7.0j:sum]
# C = abcd[20j:40j,0.0j:4.0j:sum]
# D = abcd[40j:80j,7.0j:100j:sum]
# E = abcd[40j:80j,4.0j:7.0j:sum]
# F = abcd[40j:80j,0.0j:4.0j:sum]
# G = abcd[80j:500j,7.0j:100j:sum]
# H = abcd[80j:500j:2j,4.0j:7.0j:sum]
# SR = abcd[80j:500j:2j,0.0j:4.0j:sum]

# A = abcd[20j:40j,0.3j:1.0j:sum]
# B = abcd[20j:40j,j:7.0j:sum]
# C = abcd[20j:40j,0.0j:4.0j:sum]
# D = abcd[40j:80j,7.0j:100j:sum]
# E = abcd[40j:80j,4.0j:7.0j:sum]
# F = abcd[40j:80j,0.0j:4.0j:sum]
# G = abcd[80j:500j,7.0j:100j:sum]
# H = abcd[80j:500j:2j,4.0j:7.0j:sum]
# SR = abcd[80j:500j:2j,0.0j:4.0j:sum]

# 2D_ntracks_SUEP_pt_avg_b_IRM
# A = abcd[50j:100j,7.0j:100j:sum]
# B = abcd[50j:100j,4.0j:7.0j:sum]
# C = abcd[50j:100j,0.0j:4.0j:sum]
# D = abcd[100j:150j,7.0j:100j:sum]
# E = abcd[100j:150j,4.0j:7.0j:sum]
# F = abcd[100j:150j,0.0j:4.0j:sum]
# G = abcd[150j:500j,7.0j:100j:sum]
# H = abcd[150j:500j:4j,4.0j:7.0j:sum]
# SR = abcd[150j:500j:4j,0.0j:4.0j:sum]

# 2D_SUEP_S1_ntracks_IRM
# A = abcd[0.35j:0.4j:sum,50.0j:100j:2j]
# B = abcd[0.35j:0.4j:sum,100j:150j:2j]
# C = abcd[0.35j:0.4j:sum,150j:300j:2j]
# D = abcd[0.4j:0.5j:sum,50.0j:100j:2j]
# E = abcd[0.4j:0.5j:sum,100j:150j:2j]
# F = abcd[0.4j:0.5j:sum,150j:300j:4j]
# G = abcd[0.5j:1.0j:sum,50.0j:100j:2j]
# H = abcd[0.5j:1.0j:sum,100j:150j:2j]
# SR = abcd[0.5j:1.0j:sum,150j:300j:4j]

# 2D_SUEP_S1_SUEP_nconst_IRM
# A = abcd[0.35j:0.4j:sum,10.0j:20j:2j]
# B = abcd[0.35j:0.4j:sum,20j:40j:2j]
# C = abcd[0.35j:0.4j:sum,40j:300j:2j]
# D = abcd[0.4j:0.5j:sum,10.0j:20j:2j]
# E = abcd[0.4j:0.5j:sum,20j:40j:2j]
# F = abcd[0.4j:0.5j:sum,40j:300j:2j]
# G = abcd[0.5j:1.0j:sum,10.0j:20j:2j]
# H = abcd[0.5j:1.0j:sum,20j:40j:2j]
# SR = abcd[0.5j:1.0j:sum,40j:300j:2j]

# 2D_SUEP_S1_SUEP_nconst_CL
A = abcd[0.35j:0.4j:sum,10.0j:40j:2j]
B = abcd[0.35j:0.4j:sum,40j:70j:2j]
C = abcd[0.35j:0.4j:sum,70j:300j:2j]
D = abcd[0.4j:0.5j:sum,10.0j:40j:2j]
E = abcd[0.4j:0.5j:sum,40j:70j:2j]
F = abcd[0.4j:0.5j:sum,70j:300j:2j]
G = abcd[0.5j:1.0j:sum,10.0j:40j:2j]
H = abcd[0.5j:1.0j:sum,40j:70j:2j]
SR = abcd[0.5j:1.0j:sum,70j:300j:2j]

# SR_exp = H * H.sum().value **3 * (G.sum().value * C.sum().value / A.sum().value) * \
#         ((F.sum().value / E.sum().value)**4) \
#         * (G.sum().value * F.sum().value / D.sum().value)**-2 \
#         * (H.sum().value * C.sum().value / B.sum().value)**-2

SR_exp = F * F.sum().value**3 * (G.sum().value * C.sum().value / A.sum().value) * \
        ((H.sum().value / E.sum().value)**4) \
        * (G.sum().value * F.sum().value / D.sum().value)**-2 \
        * (H.sum().value * C.sum().value / B.sum().value)**-2

print(SR_exp.sum().value/SR.sum().value)

fig = plt.figure(figsize=(10, 8))
main, sub = SR.plot_ratio(SR_exp,
                rp_num_label='SR observed',
                rp_denom_label='SR expected',
                rp_uncert_draw_type="bar")
fig.axes[0].set_yscale("log")
fig.axes[0].set_xlim(70,150)
fig.axes[1].set_ylim(0.4,1.6)
plt.savefig(output_label+"ABCD_9regions_"+plot_label+file_ext)

In [ ]:
x = 'ABCDEFGH'
y = 'ADGBEHCF'
_ = plot_ratio_regions(plots, 'A_SUEP_S1_CL', 'QCD_HT', 'data', x)

### Compare Samples
Check ABCD regions for different samples.

In [ ]:
plot_label = '2D_SUEP_S1_ntracks_IRM'
var_label = '$Sph_1$'
sample1 = 'QCD_HT'
sample2 = 'data'
density = False

abcd = plots[sample1][plot_label]
A = abcd[0.35j:0.5j,0j:100j:sum]
B = abcd[0.35j:0.5j,100j::sum]
C = abcd[0.5j:1.0j,0j:100j:sum]
D_exp = C * B.sum().value / A.sum().value

abcd2 = plots[sample2][plot_label]
A2 = abcd2[0.35j:0.5j,0j:100j:sum]
B2 = abcd2[0.35j:0.5j,100j::sum]
C2 = abcd2[0.5j:1.0j,0j:100j:sum]
D_exp2 = C2 * B2.sum().value / A2.sum().value

hists, hists2  = [A,B,C,D_exp], [A2, B2, C2, D_exp2]
labels = ['A','B','C','D_exp']
for h1, h2, label in zip(hists, hists2, labels):
        
    if density:
        h1 /= h1.sum().value
        h2 /= h2.sum().value
        
        fig, axs, res = plot_ratio(h1, h2, var_label, sample1, sample2)
        fig.suptitle(label)
        fig.subplots_adjust(top=0.92)
        axs[1].set_ylim(0.8,1.2)
        
    else:
        fig = plt.figure(figsize=(10, 8))
        _ = h1.plot_ratio(h2,
                         rp_num_label='QCD - HT Binned',
                         rp_denom_label=sample2)

        fig.suptitle(label)
        fig.axes[0].set_yscale("log")
        fig.axes[0].set_xlim(0.0,1)
        fig.axes[1].set_xlabel(var_label)
        fig.axes[1].set_ylim(0.8,1.2)
    plt.savefig(output_label + plot_label + label + file_ext, bbox_inches='tight')
    plt.show()

### Residuals

In [ ]:
plot_label = '2D_SUEP_S1_ntracks_IRM'
var_label = 'S1'
sample1 = 'QCD_HT'
sample2 = 'data'

abcd = plots[sample1][plot_label]
A = abcd[0.35j:0.5j,0j:100j:sum]
B = abcd[0.35j:0.5j,100j::sum]
C = abcd[0.5j:1.0j,0j:100j:sum]
D_exp = C * B.sum().value / A.sum().value

abcd2 = plots[sample2][plot_label]
A2 = abcd2[0.35j:0.5j,0j:100j:sum]
B2 = abcd2[0.35j:0.5j,100j::sum]
C2 = abcd2[0.5j:1.0j,0j:100j:sum]
D_exp2 = C2 * B2.sum().value / A2.sum().value

hists, hists2  = [A,B,C,D_exp], [A2, B2, C2, D_exp2]
labels = ['A','B','C','D_exp']
avgs, errs = [], []
for h1, h2, label in zip(hists, hists2, labels):
    
    h1 /= h1.sum().value
    h2 /= h2.sum().value
    fig, _, res = plot_ratio(h1, h2, var_label, sample1, sample2)
    plt.close()
    
    residuals = res[0]
    residual_stddevs = []
    for e1, e2 in zip(res[1][0], res[1][1]):
        if np.isinf(e1) and np.isinf(e2):
            print("well well well")
        elif np.isinf(e2):
            residual_stddevs.append(e1)
        elif np.isinf(e1):
            residual_stddevs.append(e1)
        else:
            residual_stddevs.append(max(e1,e2))
    residual_vars = np.array(residual_stddevs)**2
    residuals = abs(residuals[~np.isnan(residual_vars)])
    residual_vars = residual_vars[~np.isnan(residual_vars)]
    
    # Using Hist's fancy tools
    # wmean.value equivalent to np.sum(residuals/residual_vars) / np.sum(1/residual_vars)
    wmean = hist.accumulators.WeightedMean().fill(residuals, weight=1/residual_vars)
    
    # why is this difference from np.sqrt(wmean.variance)
    #test2 = 1/np.sqrt(np.sum(1/residual_vars))
    #print(test2, np.sqrt(wmean.variance))
    
    avgs.append(wmean.value)
    errs.append(np.sqrt(wmean.variance))
    
from IPython.display import display, Latex
for i in range(len(labels)):
    display(Latex(labels[i] + ' ' + str(round(avgs[i],3)) + '$\pm$' + str(round(errs[i],3))))

## Calculate Yields
Integrate histograms to calculate yields.<br>
Parameters:<br>
`lumi`: _float_, scales MC data.<br>
`plot_label`: _str_, which plot you want to integrate.

In [ ]:
plot_label = 'SUEP_nconst_CL'
samples = ['QCD_HT']
samples += ['SUEP-m125-generic', 'SUEP-m400-generic',
           'SUEP-m750-generic', 'SUEP-m1000-generic']
samples += ['SUEP-m125-darkPho', 'SUEP-m400-darkPho',
           'SUEP-m750-darkPho', 'SUEP-m1000-darkPho']
samples += ['SUEP-m125-darkPhoHad', 'SUEP-m400-darkPhoHad',
           'SUEP-m750-darkPhoHad', 'SUEP-m1000-darkPhoHad']

for sample in samples:
    h = plots[sample][plot_label]
    passing = h[70j:].sum().value
    total = h.sum().value
    print(sample, round(passing/total, 3))

## Table

Make comparison table by running next cells. The output is meant to be in Latex format.

In [ ]:
selection = 'ch'

for plot_label in ['A','B','C','D_exp']:
    
    plot_label+="_"+selection
    
    # QCD hist for ABCD region
    hdata = plots['data'][plot_label]
    hQCD_MC = plots['QCD_MC'][plot_label]
        
    # fill these up for the table
    stats['data'][plot_label]["value"] = hdata.sum().value
    stats['data'][plot_label]["err"] = hdata.sum().variance
    stats['QCD_MC'][plot_label]["value"] = hQCD_MC.sum().value
    stats['QCD_MC'][plot_label]["err"] = hQCD_MC.sum().variance

In [ ]:
table = []
for i in range(len(list(stats.keys()))):
    label = list(stats.keys())[i]
    line = []
    line.append(label)
    
    for r in ['A_'+selection,'B_'+selection,'C_'+selection,'D_exp_'+selection,'D_obs_'+selection]:
        if 'QCD_MC' in label:
            line.append("{:0.0f} ".format(stats[label][r]['value']) + r'$\pm$' + " {:0.0f}".format(stats[label][r]['err']))
        else:
            line.append("{:0.2f} ".format(stats[label][r]['value']) + r'$\pm$' + " {:0.2f}".format(stats[label][r]['err']))
        
    if 'QCD_MC' in label:
        line.append(round((stats[label]['D_exp_'+selection]['value']-stats[label]['D_obs_'+selection]['value'])/stats[label]['D_exp_'+selection]['value'],2))
        line.append(0)
    else:
        line.append(0)
        line.append(stats[label]['D_obs_'+selection]['value']/math.sqrt(stats[label]['D_obs_'+selection]['value']+stats[label]['D_exp_'+selection]['value']+stats[label]['D_obs_'+selection]['err']+stats['QCD_MC']['D_exp_'+selection]['err']))
    table.append(line)                                                         

In [ ]:
t = tabulate(table,tablefmt ='latex',
               headers=['Process', 'A', 'B', 'C', 'D_{exp}', 'D_{obs}', 'D_{exp}-D_{obs}/D_{exp}', 'S/$\sqrt(S+B+\sigma_{S}+\sigma_{B}$)'])
with open('table.txt', 'w') as f:
    f.write(t)
t

In [ ]:
def convert2LatexString(s):
    r = r'(\^\{\})'; s = re.sub(r, "^", s)
    s = re.sub(r'\\([\$\_\{\}\^])', r'\1', s)
    s = re.sub(r'(\\textbackslash{})', r'\\', s)
    return s

t = convert2LatexString(t)
with open('table.txt', 'w') as f:
    f.write(t)
t